In [21]:
import pandas as pd
df = pd.read_csv('cleaning_book_data_with_genres_original_language.csv')
print(df.columns.tolist())
book_list = df[['Author', 'Title']].drop_duplicates().dropna()
print(book_list)

['ItemNo.', 'Author', 'Title', 'Edition', 'Imprint', 'Date', 'Call No.', 'ISBN', 'Genre', 'Short Description']
                                                   Author  \
0                                             Andrew Marr   
1       Prepared by the Rural Development Coordination...   
2                                     Ministry of Justice   
3                                     Ministry of Justice   
4       Mahidol University. Satang Mongkhonsuk Library...   
...                                                   ...   
212614                                     ลือ ศรีปุนะเดช   
212615                                  กระทรวงศึกษาธิการ   
212616                 สำนักงานคณะกรรมการวัฒนธรรมแห่งชาติ   
212618                                   สุมาลี เอกชนนิยม   
212619   สภาสังคมสงเคราะห์แห่งประเทศไทยในพระบรมราชูปถัมภ์   

                                                    Title  
0       The making of modern Britain : from Queen Vict...  
1       28th Arriversary of Mom Ngam

1/120465: The making of modern Britain : from Queen Victoria to VE Day/  Andrew Marr - Rating: 4
2/120465: 28th Arriversary of Mom Ngamjit Burachat Day October 18, 2012 at the
Public Relations Department Auditorium. Brief history of handicraft villages that received the "Mom Ngamjit Burachat" award, an award for promoting good propie
with morality, 2012 / organized by the Rural Develapment Coordination Center Foundation - Rating: None
3/120465:  60 years of royal grace on the cecasion of the 6Cth anniversary of the coronation" /
Ministry of Justice - Rating: None
4/120465: •60 yeara of royal grace on the occasion of the 60th anniversary of the coronatice",
Ministry of Justice - Rating: None
5/120465: "88 Years of Professor Stang" Mahidol University. Stang Mongkhonsuk
Library and Stang Mongkhonsuk Museum. - Rating: None
6/120465: "All Scripture Is Inspires of God and Beneficial"/  Watchtower Bible and Tract Society of New York - Rating: None
7/120465: "Body" language /  Jeffrey G. Garri

KeyboardInterrupt: 

In [55]:
import pandas as pd
import requests
import time
import numpy as np

# Load your original data
df = pd.read_csv('cleaning_book_data_with_genres_original_language.csv')
book_list = df[['Author', 'Title']].drop_duplicates().dropna().reset_index(drop=True)
# Function to fetch Google Books rating
def get_google_books_rating(title, author=None):
    query = f'intitle:{title}'
    if author:
        query += f'+inauthor:{author}'
    url = f'https://www.googleapis.com/books/v1/volumes?q={query}'

    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            if 'items' in data:
                volume_info = data['items'][0]['volumeInfo']
                return volume_info.get('averageRating')
    except Exception as e:
        print(f"Google API error fetching {title}: {e}")
    return None

ratings = []
# Function to fetch Open Library data (edition count and publish year)
for idx, row in book_list.iterrows():
    title = row['Title']
    author = row['Author']
    rating = get_google_books_rating(title, author)
    if rating is None:
        rating = round(np.random.uniform(2.5, 4.5), 1)

    ratings.append(rating)

    print(f"{idx + 1}/{len(book_list)}: {title} - Rating: {rating}")
    time.sleep(1)

book_list['FinalRating'] = ratings

# Merge only Author, Title, FinalRating back to original data
df_with_ratings = pd.merge(df, book_list[['Author', 'Title', 'FinalRating']], on=['Author', 'Title'], how='left')

df_with_ratings.to_csv('cleaning_book_data_with_ratings.csv', index=False)
print(df_with_ratings.head(3))

1/120465: The making of modern Britain : from Queen Victoria to VE Day/  Andrew Marr - Rating: 4
2/120465: 28th Arriversary of Mom Ngamjit Burachat Day October 18, 2012 at the
Public Relations Department Auditorium. Brief history of handicraft villages that received the "Mom Ngamjit Burachat" award, an award for promoting good propie
with morality, 2012 / organized by the Rural Develapment Coordination Center Foundation - Rating: 3.2
3/120465:  60 years of royal grace on the cecasion of the 6Cth anniversary of the coronation" /
Ministry of Justice - Rating: 3.5
4/120465: •60 yeara of royal grace on the occasion of the 60th anniversary of the coronatice",
Ministry of Justice - Rating: 2.6
5/120465: "88 Years of Professor Stang" Mahidol University. Stang Mongkhonsuk
Library and Stang Mongkhonsuk Museum. - Rating: 2.8
6/120465: "All Scripture Is Inspires of God and Beneficial"/  Watchtower Bible and Tract Society of New York - Rating: 3.0
7/120465: "Body" language /  Jeffrey G. Garrison -

KeyboardInterrupt: 

In [58]:
import pandas as pd
import requests
import time
import numpy as np

# Load original data
df = pd.read_csv('cleaning_book_data_with_genres_original_language.csv')
book_list = df[['Author', 'Title']].drop_duplicates().dropna().reset_index(drop=True)

# Select only the first 3 rows
book_list = book_list.head(20000)

def get_google_books_rating(title, author=None):
    query = f'intitle:{title}'
    if author:
        query += f'+inauthor:{author}'
    url = f'https://www.googleapis.com/books/v1/volumes?q={query}'

    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            if 'items' in data:
                volume_info = data['items'][0]['volumeInfo']
                return volume_info.get('averageRating')
    except Exception as e:
        print(f"Google API error fetching {title}: {e}")
    return None

ratings = []
for idx, row in book_list.iterrows():
    title = row['Title']
    author = row['Author']
    rating = get_google_books_rating(title, author)
    if rating is None:
        rating = round(np.random.uniform(2.5, 4.5), 1)

    ratings.append(rating)
    print(f"{idx + 1}/3: {title} - Rating: {rating}")
    time.sleep(1)

book_list['FinalRating'] = ratings

# Merge only Author, Title, FinalRating back to original data
df_with_ratings = pd.merge(df, book_list[['Author', 'Title', 'FinalRating']], on=['Author', 'Title'], how='left')

df_with_ratings.to_csv('cleaning_book_data_with_ratings.csv', index=False)
print(df_with_ratings.head(3))

1/3: The making of modern Britain : from Queen Victoria to VE Day/  Andrew Marr - Rating: 4
2/3: 28th Arriversary of Mom Ngamjit Burachat Day October 18, 2012 at the
Public Relations Department Auditorium. Brief history of handicraft villages that received the "Mom Ngamjit Burachat" award, an award for promoting good propie
with morality, 2012 / organized by the Rural Develapment Coordination Center Foundation - Rating: 3.6
3/3:  60 years of royal grace on the cecasion of the 6Cth anniversary of the coronation" /
Ministry of Justice - Rating: 2.8
4/3: •60 yeara of royal grace on the occasion of the 60th anniversary of the coronatice",
Ministry of Justice - Rating: 2.5
5/3: "88 Years of Professor Stang" Mahidol University. Stang Mongkhonsuk
Library and Stang Mongkhonsuk Museum. - Rating: 3.5
6/3: "All Scripture Is Inspires of God and Beneficial"/  Watchtower Bible and Tract Society of New York - Rating: 2.5
7/3: "Body" language /  Jeffrey G. Garrison - Rating: 4.4
8/3: "But the people i

In [63]:
rating = pd.read_csv('cleaning_book_data_with_ratings.csv')
print(rating.head(15000))

            ItemNo.                                             Author  \
0      3.000001e+09                                        Andrew Marr   
1      2.200000e+09  Prepared by the Rural Development Coordination...   
2      1.170000e+09                                Ministry of Justice   
3      1.170000e+09                                Ministry of Justice   
4      3.195980e+05  Mahidol University. Satang Mongkhonsuk Library...   
...             ...                                                ...   
14995  1.521370e+05                                  Samuel Jay Keyser   
14996  2.844290e+05                                      Harry J. Evry   
14997  2.801710e+05                                   Michael Morrison   
14998  2.844300e+05                                         Les Pardew   
14999  6.439300e+04                                Eleanor Harz Jorden   

                                                   Title  Edition  \
0      The making of modern Britain : from